In [38]:
import pandas as pd


subreddit = "nosleep"
oldpath = f'./reddit/{subreddit}_2024_09_30.json'
data = pd.read_json(oldpath)

def clean_data(df):
    def get_flair(flair_richtext):
        import ast
        flair = ast.literal_eval(flair_richtext)
        return flair[0]['t']

    df = df.copy()
    remain_col = ["title", "link_flair_richtext", "author", "url", "selftext"]
    new_df = df[remain_col]
    # new_df["link_flair_richtext"] = new_df["link_flair_richtext"].apply(get_flair)
    

    new_df.columns = ["title", "flair", "author", "url", "selftext"]

    url = new_df["url"]
    new_df.drop(columns=["url"], inplace=True)
    new_df.insert(1, "url", url)

    return new_df
new_data = clean_data(data)
new_data.to_json(oldpath, orient='records', lines=True, force_ascii=False)

C:\Users\dangm\AppData\Local\Temp\ipykernel_14140\2135765595.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(columns=["url"], inplace=True)


`title` 
`link_flair_richtext`
`author`
`url`
`selftext`

In [35]:
for col in data.columns:
        print(col)

approved_at_utc
subreddit
selftext
author_fullname
saved
mod_reason_title
gilded
clicked
title
link_flair_richtext
subreddit_name_prefixed
hidden
pwls
link_flair_css_class
downs
thumbnail_height
top_awarded_type
hide_score
name
quarantine
link_flair_text_color
upvote_ratio
author_flair_background_color
subreddit_type
ups
total_awards_received
media_embed
thumbnail_width
author_flair_template_id
is_original_content
user_reports
secure_media
is_reddit_media_domain
is_meta
category
secure_media_embed
link_flair_text
can_mod_post
score
approved_by
is_created_from_ads_ui
author_premium
thumbnail
edited
author_flair_css_class
author_flair_richtext
gildings
content_categories
is_self
mod_note
created
link_flair_type
wls
removed_by_category
banned_by
author_flair_type
domain
allow_live_comments
selftext_html
likes
suggested_sort
banned_at_utc
url_overridden_by_dest
view_count
archived
no_follow
is_crosspostable
pinned
over_18
all_awardings
awarders
media_only
link_flair_template_id
can_gild
sp

In [33]:
data['url'][1]

'https://www.reddit.com/r/creepypasta/comments/1dctbhv/post_creepy_images_on_reyescream_our_new_subreddit/'

In [36]:
data['selftext']

0                                                    NaN
1      Hi, Pasta Aficionados!\n\nLet's talk about r/E...
2      So me, my parents and my sister, live on a far...
3                         read it for the first time. :(
4      Looking for a longer story, not too long with ...
                             ...                        
960    In the early 90's a close childhood friend of ...
961    I remember years ago Red Mist being a similar ...
962    I don’t know why I took the detour that night....
963    \n\n“Attention, teachers and students, recess ...
964    I lost my old phone and along with it a video ...
Name: selftext, Length: 965, dtype: object